In [1]:
import math
import numpy as np
import pandas as pd
import csv
import statistics as stat
import matplotlib.pyplot as plt
import os
import glob
from datetime import date
from datetime import datetime
from datetime import timedelta  

#  Pt 1 - Date Generator 

In [2]:
# input start and end date for the input files
start_date = '2009-01-01'
end_date   = '2017-01-01'

s_date = datetime.strptime(start_date, '%Y-%m-%d')
e_date = datetime.strptime(end_date, '%Y-%m-%d')
dates = pd.date_range(s_date, e_date)

In [12]:
# currently using Greg's files to get a set of all buno's that are FA-18, but can be easily replaced by any other file
buno = np.load('bunos.npy')
bunoDF = pd.DataFrame(buno)
bunoDF.rename(columns={0:'BUNO'},inplace=True)

In [4]:
# create daily entries for within start and end date for each Buno
df = list()
for b in bunoDf:
    for d in dates:
        df.append([d, b])
df = pd.DataFrame(df, columns=['Date', 'Buno'])

In [5]:
# save to csv file
df.to_csv(r'dates.csv', index=False)

# Pt 2 - Maintenance Wrangling

In [25]:
list(glob.iglob(os.path.join('./maint_data', '*.txt')))

['./maint_data/test.txt']

In [30]:
#import all maintenance files
#read 
#must be CSV
#input: list of files to be read in 
def read_in_files(file_type):
    if file_type == 'maintenance':
        directory = 'maint_data'
    elif file_type == 'sharp':
        directory = 'sharp_data'
    else:
        return 'Invalid file type: must be maintenance or sharp'
    
    file_names = list(glob.iglob(os.path.join('./'+directory, '*.csv')))
    frames=[]
    for file_name in file_names:
        frames.append(pd.read_csv(open(file_name, 'r', errors='ignore'), dtype={'Bu/SerNo': str, 'Position Code': str}, low_memory=False))

    maintDf = pd.concat(frames)
    return maintDf


maintDf = read_in_files('maintenance')

In [7]:
# grab pertinant columns
cols = ['Bu/SerNo', 'Maint Level', 'Type Maint Code', 'Action Taken', 'Manhours', 'Rcvd Date', 'Rmvd PartNo', 'Rcvd EOC Code']

## <span style="color:#808080">Derive MC Status  </span>

In [8]:
# method for derviving mc status based off EOC code and maintanence level
# 2 is FMC
# 1 is PMC
# 0 is NMC

def deriveMC(df):
    mcStatus = list()
    for index, row in df.iterrows():
        AtoB = list(map(chr, range(ord('A'), ord('B')+1)))
        CtoX = list(map(chr, range(ord('C'), ord('X')+1)))
        YtoZ = list(map(chr, range(ord('Y'), ord('Z')+1)))
        code = row['Rcvd EOC Code']
        if(pd.isnull(code)):
            mcStatus.append(2)
            continue
            
        code = str(code)
        code = code.upper()
        code = code.strip()

        if (code == "" or code in AtoB) and row['Maint Level'] != 3:
            mcStatus.append(2)
        elif code in CtoX:
            mcStatus.append(1)
        elif code in YtoZ or row['Maint Level'] == 3:
            mcStatus.append(0)
        else:
            print('Error on row %d' % index)
    return mcStatus

# change date to pandas timestamp
def changeDate(df):
    date = list()
    for index, row in df.iterrows():
        date.append(pd.Timestamp(row['Rcvd Date']))
    return date

# if data has multiple entries and different MC status, pick the lowest one
def diffSameDayEntry(df):
    prev = df.iloc[0]
    mc = list()
    for index, row in df.iterrows():
        mc.append(row['MC'])
        if index == 0:
            continue
        if prev['Buno'] == row['Buno'] and prev['Date'] == row['Date']:
            lowMC = min(mc[-2], mc[-1])
            mc[-2] = lowMC
            mc[-1] = lowMC
        prev = row
    return mc

In [9]:
maintDf['MC'] = deriveMC(maintDf)
maintDf['Rcvd Date'] = changeDate(maintDf)
cols += ['MC']

# use data that is only unscheduled
mcStatus = maintDf[maintDf['Type Maint Code'] == 'B']

## <span style="color:#808080">Drop all duplicate entries & set lowest mc status for different mc entries </span>

In [10]:
mcStatus = mcStatus[['Bu/SerNo', 'Rcvd Date', 'MC']].drop_duplicates()
mcStatus.rename(columns={'Bu/SerNo': 'Buno', 'Rcvd Date': 'Date'}, inplace=True)
mcStatus = mcStatus.sort_values(['Buno', 'Date'], ascending=[True, True])
mcStatus = mcStatus.reset_index()
mcStatus = mcStatus.drop(columns=['index'])

# dropping all same day entries that have different mc status
mcStatus['MC'] = diffSameDayEntry(mcStatus)
mcStatus = mcStatus.drop_duplicates()
mcStatus['MC'] = diffSameDayEntry(mcStatus)
mcStatus = mcStatus.drop_duplicates()
len(mcStatus)

594644

In [11]:
# save to csv file
mcStatus.to_csv(r'BunoMC.csv', index=False)

# Pt 3 - MC Date Merger

In [12]:
dfEvent = pd.read_csv('BunoMC.csv')
dfEvent = dfEvent[['Buno', 'Date', 'MC']]

dates = pd.read_csv('Dates.csv')

C:\Users\lawsh\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## <span style="color:#808080">Change dateframe to pandas timeframe to merge </span>

In [13]:
def changeDate(df):
    d = list()
    for i in df['Date']:
        d.append(pd.Timestamp(i))
    return d
def changeBuno(df):
    b = list()
    for i in df['Buno']:
        b.append(str(i))
    return b

In [14]:
dfEvent['Date'] = changeDate(dfEvent)
dates['Date'] = changeDate(dates)
dates['Buno'] = changeBuno(dates)

df = pd.merge(dates, dfEvent, on=['Buno', 'Date'], how='left')

## <span style="color:#808080">Populate daily entries with MC status  </span>

In [15]:
event = []
event.append(df['MC'].iloc[0])
prev = df['Buno'].iloc[0]
for index, row in df.iterrows():
    if index == 0:
        continue
    event.append(row['MC'])
    if pd.isnull(row['MC']) and prev == row['Buno']:
        event[-1] = event[-2]
    prev = row['Buno']
df['MC'] = event
df = df.dropna()
df = df.reset_index()
df = df.drop(['index'], axis=1)

In [16]:
df.to_csv(r'DailyBunoMC.csv', index=False)

# Pt 4 - Sharp Analysis

In [17]:
sharpDf = pd.read_csv('/sharp_data/sharp.csv', low_memory=False)

## <span style="color:#808080">Grab only important columns</span>

In [18]:
c = ['LaunchDate', 'Buno','TFT', 'Land ICAO', 'Pilot Rank']
tmr = list()
for i in range(1,12):
    tmr.append('TMR%d' % i)
c += tmr
sharpDf = sharpDf.sort_values(['Buno', 'LaunchDate']).reset_index()[c]

## <span style="color:#808080">See if plane landed on ship  </span>

In [19]:
ship = list()
for index, row in sharpDf.iterrows():
    if 'USS' in row['Land ICAO']:
        ship.append(1)
    else:
        ship.append(0)
sharpDf['Ship'] = ship
sharpDf = sharpDf.drop(columns=['Land ICAO'])

## <span style="color:#808080">Calculate number of flights in the same day </span>

In [20]:
# make list that calculates total flights flown in that day
flights = list()
prev = sharpDf.iloc[0]
counter = 1
for index, row in sharpDf.iterrows():
    if index == 0:
        flights.append(counter)
        continue
    if prev['Buno'] == row['Buno'] and prev['LaunchDate'] == row['LaunchDate']:
        counter += 1
    else:
        counter = 1
    flights.append(counter)
    prev = row

# make each day have the same total flights flown
flights.reverse()
sharpDf = sharpDf.iloc[::-1].reset_index()

prev = sharpDf.iloc[0]
for index, row in sharpDf.iterrows():
    if index == 0:
        continue
    if prev['Buno'] == row['Buno'] and prev['LaunchDate'] == row['LaunchDate']:
        flights[index] = flights[index-1]
    prev = row

# add into sharpDf
flights.reverse()
sharpDf = sharpDf.iloc[::-1].reset_index()
sharpDf['Flights'] = flights
c += ['Flights']

## <span style="color:#808080"> One Hot Encode  </span>

In [21]:
# create one hot for all tmr tags
one_hot = pd.get_dummies(sharpDf['TMR1'])
for index, row in sharpDf.iterrows():
    for i in range(2,12):
        tag = sharpDf['TMR%d' % i].iloc[index]
        if pd.isnull(tag):
            break
        one_hot.at[index, tag] = one_hot[tag].iloc[index] + 1

# combine all same day tags and add up total flight time
prev = sharpDf.iloc[0]
FT = list()
for index, row in sharpDf.iterrows():
    if index == 0:
        FT.append(row['TFT'])
        continue
    if prev['Buno'] == row['Buno'] and prev['LaunchDate'] == row['LaunchDate']:
        for i in range(1,12):
            tag = sharpDf['TMR%d' % i].iloc[index-1]
            if pd.isnull(tag):
                break
            one_hot.at[index, tag] += one_hot[tag].iloc[index-1]
        FT.append(FT[-1] + sharpDf['TFT'].iloc[index])
    else:
        FT.append(sharpDf['TFT'].iloc[index])
    prev = row


sharpDf['TFT'] = FT
sharpDf.drop(columns=tmr, inplace=True)
sharpDf = pd.concat([sharpDf, one_hot], axis=1)
sharpDf.drop(['level_0', 'index'], axis=1, inplace=True)

## <span style="color:#808080">Drop all duplicate days </span>

In [22]:
sharpDf = sharpDf.iloc[::-1].reset_index()
sharpDf = sharpDf.drop(['index'], axis=1)
df = list()
prev = sharpDf.iloc[0]
for index, row in sharpDf.iterrows():
    df.append(row)
    if index == 0:
        continue
    if prev['Buno'] == row['Buno'] and prev['LaunchDate'] == row['LaunchDate']:
        df.pop()
    prev = row
df = pd.DataFrame(df)
len(df)

650566

## <span style="color:#808080">Give numeric value to each pilot rank </span>

In [23]:
ranks = {'nan': 0,
 'CIVILIAN': 1,
 'CIVILIAN (CONTRACTOR)': 2,
 'E-1': 3,
 'E-6': 4,
 'E-7': 5,
 'MIDSHIPMAN': 6,
 'O-1': 7,
 'O-2': 8,
 'O-3': 9,
 'O-4': 10,
 'O-5': 11,
 'O-6': 12,
 'O-7': 13,
 'O-8': 14,
 'O-9': 15,
 'O-10': 16}
p_rank = list()
for index, row in df.iterrows():
    if pd.isnull(row['Pilot Rank']):
        p_rank.append(0)
    else:
        p_rank.append(ranks[row['Pilot Rank']])
df['Pilot Rank'] = p_rank
df.head()

,LaunchDate,Buno,TFT,Pilot Rank,Ship,Flights,1A0,1A1,1A2,1A3,...,6Y1,7N0,7N1,7O1,7O3,7O6,7T9,7W2,7X2,7X7
0,04/28/2009,999998,2.6,9,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,01/07/2010,999998,1.5,9,0,2,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
3,01/06/2009,999998,1.5,9,0,3,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
6,04/21/2011,999961,2.2,11,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,03/17/2011,999961,0.5,10,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## <span style="color:#808080"> Read in mc status </span>

In [24]:
mcDf = pd.read_csv('DailyBunoMC.csv')

In [25]:
def convertTime(x, tag):
    date = list()
    for index, row in x.iterrows():
        date.append(pd.Timestamp(row[tag]))
    return date

In [26]:
# convert to pandas timestamp
mcDf['Date'] = convertTime(mcDf, 'Date')
df['LaunchDate'] = convertTime(df, 'LaunchDate')

In [27]:
# grab next day mc status by merging mcDf on previous day
date = list()
for index, row in mcDf.iterrows():
    date.append(row['Date'] + pd.Timedelta(days=-1))
mcDf['LaunchDate'] = date
mcDf.head()

,Date,Buno,MC,LaunchDate
0,2009-01-14,165167,2.0,2009-01-13
1,2009-01-15,165167,2.0,2009-01-14
2,2009-01-16,165167,2.0,2009-01-15
3,2009-01-17,165167,2.0,2009-01-16
4,2009-01-18,165167,2.0,2009-01-17


## <span style="color:#808080">  Merge dataframes </span>

In [28]:
df = df.merge(mcDf, on=['LaunchDate', 'Buno'], how='inner')
mcDf['LaunchDate'] = mcDf['Date']
df = df.merge(mcDf, on=['LaunchDate', 'Buno'], how='inner')
df = df.drop(['Date_x', 'Date_y'], axis=1)
df.head()

,LaunchDate,Buno,TFT,Pilot Rank,Ship,Flights,1A0,1A1,1A2,1A3,...,7N1,7O1,7O3,7O6,7T9,7W2,7X2,7X7,MC_x,MC_y
0,2016-12-16,169123,1.0,9,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,2.0,2.0
1,2016-12-09,169123,0.6,9,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0.0
2,2016-12-08,169123,1.9,9,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0.0
3,2016-12-07,169123,0.9,11,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0.0
4,2016-12-05,169123,3.6,9,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0.0


In [29]:
# see if the plane was damaged next day after flight
# MC_x is next day's MC 
# MC_y is that day's MC

dam = list()
for index, row in df.iterrows():
    if row['MC_x'] < row['MC_y']:
        dam.append(1)
    else:
        dam.append(0)
df['dam'] = dam
flightDf = df

In [30]:
fields = df.drop(['LaunchDate', 'Buno', 'MC_x', 'MC_y', 'Pilot Rank'], axis=1).columns

In [31]:
df.head()

,LaunchDate,Buno,TFT,Pilot Rank,Ship,Flights,1A0,1A1,1A2,1A3,...,7O1,7O3,7O6,7T9,7W2,7X2,7X7,MC_x,MC_y,dam
0,2016-12-16,169123,1.0,9,0,1,0,0,0,0,...,0,0,0,0,0,0,0,2.0,2.0,0
1,2016-12-09,169123,0.6,9,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
2,2016-12-08,169123,1.9,9,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
3,2016-12-07,169123,0.9,11,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
4,2016-12-05,169123,3.6,9,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0


In [32]:
df.shape

(167557, 225)

In [33]:
df.to_csv(r'Sharp_cleaned.csv', index=False)